In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
from meteostations.clients import agrometeo, netatmo
from tstore.extensions.array import TSArray
from tstore.ts import TS, TSDFPandas

ERROR 1: PROJ: proj_create_from_database: Open of /home/martibosch/mambaforge/envs/meteostations-vector-cube/share/proj failed
/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'TSArray' from partially initialized module 'tstore.extensions.array' (most likely due to a circular import) (/home/martibosch/data-science/meteostations-vector-cube/src/tstore/tstore/extensions/array.py)

## Preprocessing
### Netatmo

In [ ]:
netatmo_df_filepath = "../data/interim/netatmo-lausanne-aug-21/ts-df.csv"
netatmo_gser_filepath = "../data/interim/netatmo-lausanne-aug-21/station-gser.gpkg"

In [ ]:
netatmo_gser = gpd.read_file(netatmo_gser_filepath).set_index("index")["geometry"]
netatmo_gser

In [ ]:
netatmo_df = pd.read_csv(netatmo_df_filepath)
netatmo_df["time"] = pd.to_datetime(netatmo_df["time"])
netatmo_df["variable"] = netatmo_df["variable"].map(
    {v: k for k, v in netatmo.NetatmoClient._ecv_dict.items()}
)
netatmo_df

### Agrometeo

In [ ]:
region = netatmo_gser.unary_union.convex_hull
client = agrometeo.AgrometeoClient(region=region, crs=netatmo_gser.crs)

In [ ]:
agrometeo_gser = client.stations_gdf.set_index(client._stations_id_col)["geometry"]
agrometeo_gser

In [ ]:
start_dt = netatmo_df["time"].min()
end_dt = netatmo_df["time"].max()


def get_ts_df(variable):
    try:
        return (
            client.get_ts_df(variable=variable, start_date=start_dt, end_date=end_dt)
            .loc[start_dt:end_dt]
            .resample("1H")
            .mean()
            .stack()
            .rename("value")
            .reset_index()
            .assign(variable=variable)
        )
    except KeyError:
        return None


agrometeo_df = pd.concat(
    [get_ts_df(variable) for variable in netatmo_df["variable"].unique()],
).rename(columns={"name": "id"})
agrometeo_df

In [ ]:
ts_df = pd.concat([netatmo_df, agrometeo_df])
ts_df

In [ ]:
for var, var_ts_df in ts_df.groupby("variable"):
    for station_id, group_df in var_ts_df.groupby("id"):
        break
    break
group_df

In [ ]:
TS(group_df.set_index("time")[["value"]])

In [ ]:
def ts_ser_from_df(var_ts_df, station_id_col="id"):
    station_ids = []
    ts_list = []
    for station_id, group_df in var_ts_df.groupby("id"):
        station_ids.append(station_id)
        ts_list.append(TS(group_df.set_index("time")[["value"]]))
    return pd.Series(TSArray(ts_list), index=station_ids)

ts_ser_from_df(var_ts_df)

In [ ]:
ts_ser

In [ ]:
tsdf = TSDFPandas({var: ts_ser_from_df(var_ts_df) for var, var_ts_df in ts_df.groupby("variable")})
tsdf

In [ ]:
# TODO: test tsdf.to_tstore